自分のKaggleのjsonファイルをアップロードする

In [0]:
from google.colab import files
files.upload()

今回で使うファイルをKaggleからダウンロードします

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!pip install kaggle

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c digit-recognizer

    100% |████████████████████████████████| 61kB 2.1MB/s 
    100% |████████████████████████████████| 235kB 7.1MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
 94% 69.0M/73.2M [00:01<00:00, 33.0MB/s]
100% 73.2M/73.2M [00:01<00:00, 66.1MB/s]
 68% 33.0M/48.8M [00:00<00:00, 30.2MB/s]
100% 48.8M/48.8M [00:00<00:00, 68.5MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 75.9MB/s]


Chainerを使えるように必要なものをinstallします

In [0]:
!curl https://colab.chainer.org/install | sh -
 

必要なモジュールをimportします

In [0]:
import argparse
import copy
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainer import cuda, training, reporter, Variable, serializers
from chainer.training import trainer, extensions
from chainer.dataset import convert
from chainer.datasets import tuple_dataset
from chainer.dataset import iterator as iterator_module
from chainer import optimizer as optimizer_module


argparserを設定します

In [0]:
parser = argparse.ArgumentParser(description='MNIST')
parser.add_argument('--init', '-i', default=0, type=int)
parser.add_argument('--gpu', '-g', default=0, type=int)
parser.add_argument('--out', '-o', default='result')
parser.add_argument('--epoch', '-e', default=30, type=int)
parser.add_argument('--batchsize', '-b', default=100, type=int)
parser.add_argument('--file_name', '-f', default='test', type=str)
parser.add_argument('--snapshot', '-s', default=10, type=int)
parser.add_argument('--withlabel', '-wl', default=False, type=bool)
args = parser.parse_args()

データセットをロードする関数を定義します

In [0]:
def Load_Dataset(withlabel=True, conv=True):
    input = pd.read_csv("train.csv")
    data = input.values
    #画像とラベルに分ける．画像は255で割って0~1に正規化
    img = (data[:,1:]/255.0).astype(np.float32)
    label = (data[:,:1].flatten()).astype(np.int32)
    #imgを28x28にreshape
    if conv:
        img = np.reshape(img, (-1, 1, 28, 28))
    #trainとtestのしきい値を決める
    threshhold = np.int32(len(img)*0.8)
    #tupledatasetを作る
    if withlabel:
        train = tuple_dataset.TupleDataset(img[0:threshhold], label[0:threshhold])
        test = tuple_dataset.TupleDataset(img[threshhold:], label[threshhold:])
    else:
        train = tuple_dataset.TupleDataset(img[0:threshhold],)
        test = tuple_dataset.TupleDataset(img[threshhold:],)
    return train, test


Trainerで使うEvaluatorやUpdaterを作ります

In [0]:
class MyEvaluator(extensions.Evaluator):
    def __init__(self, iterator, CLS,
        converter=convert.concat_examples,device=0, eval_hook=None,
        eval_func=None):
        if isinstance(iterator, iterator_module.Iterator):
            iterator = {'main': iterator}
        self._iterators = iterator
        self._targets = {'main':CLS}

        self.converter = converter
        self.device = device
        self.eval_hook = eval_hook

    def evaluate(self):
        iterator = self._iterators['main']
        self.CLS = self._targets['main']
        xp = np if int(self.device) == -1 else cuda.cupy
        it = copy.copy(iterator)
        summary = reporter.DictSummary()
        for batch in it:
            observation = {}
            with reporter.report_scope(observation):
                input = self.converter(batch, self.device)
                x_batch = xp.array(input[0])
                t_batch = xp.array(input[1])
                self.loss = 0
                self.acc = 0
                with chainer.using_config('train', False):
                    y = self.CLS(x_batch, softmax=False)
                    self.loss = F.softmax_cross_entropy(y, t_batch)
                    self.acc = F.accuracy(y, t_batch)
                observation['val/loss'] = self.loss
                observation['val/acc'] = self.acc
            summary.add(observation)
        return summary.compute_mean()


In [0]:
class MyUpdater(training.StandardUpdater):
    def __init__(self, iterator, CLS, opt,
        converter=convert.concat_examples,device=0):
        if isinstance(iterator, iterator_module.Iterator):
            iterator = {'main':iterator}
        self._iterators = iterator
        self.CLS = CLS
        self._optimizers = {"main":opt}
        self.converter = convert.concat_examples
        self.device = device
        self.iteration = 0

    def update_core(self):
        """lossを計算"""
        iterator = self._iterators['main'].next()
        input = self.converter(iterator, self.device)
        xp = np if int(self.device) == -1 else cuda.cupy
        x_batch = xp.array(input[0])
        t_batch = xp.array(input[1])
        self.loss = 0
        self.acc = 0
        #計算開始
        y = self.CLS(x_batch, softmax=False)
        self.loss = F.softmax_cross_entropy(y, t_batch)
        self.acc = F.accuracy(y, t_batch)
        self._optimizers["main"].target.cleargrads()
        self.loss.backward()
        self._optimizers["main"].update()
        reporter.report({'main/loss':self.loss, 'main/acc':self.acc})


今回使うネットワークは畳込み3層+全結合2層です

In [0]:
class CLS(chainer.Chain):
    #入力は1チャネル白黒画像
    def __init__(self):
        super(CLS, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 8, 3, pad=1)
            self.ebn1 = L.BatchNormalization(8)
            self.conv2 = L.Convolution2D(8, 16, 3, pad=1)
            self.ebn2 = L.BatchNormalization(16)
            self.conv3 = L.Convolution2D(16, 32, 3, pad=1)
            self.fc1 = L.Linear(None, 1000)
            self.fc2 = L.Linear(1000, 10)

    def __call__(self, x, softmax=True):
        return self.cls(x, softmax)
    def cls(self, x, softmax=True):
        h = F.leaky_relu(self.ebn1(self.conv1(x)))
        h = F.leaky_relu(self.ebn2(self.conv2(h)))
        h = F.leaky_relu(self.conv3(h))
        h = F.leaky_relu(self.fc1(h))
        h = F.leaky_relu(self.fc2(h))
        if softmax:
            return F.softmax(h)
        else:
            return h


Trainerを使って学習していきます．

In [72]:
print('#GPU:{}'.format(args.gpu))
print('#minibatch-size:{}'.format(args.batchsize))
print('#epoch:{}'.format(args.epoch))
print('')

CLS.to_gpu()
print('Loading')
train, test = Load_Dataset(withlabel=True, conv=True)
#train, test = chainer.datasets.mnist.get_mnist(withlabel=True, ndim=3)
print('Loaded')
def make_optimizer(model, alpha=0.0002, beta1=0.9, beta2=0.999):
    #optimizer = chainer.optimizers.MomentumSGD(lr=0.01)
    optimizer = chainer.optimizers.Adam(alpha=alpha, beta1=beta1, beta2=beta2)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.0001))
    return optimizer
opt = make_optimizer(CLS)
#Laod the dataset
train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
test_iter = chainer.iterators.SerialIterator(test, args.batchsize,
                                            repeat=False, shuffle=False)
updater = MyUpdater(train_iter, CLS, opt, device=args.gpu)
trainer = training.Trainer(updater, (args.epoch, 'epoch'),
    out="{}/b{}".format(args.out, args.batchsize))
#trainer.extend(extensions.observe_lr())
trainer.extend(MyEvaluator(test_iter, CLS,
    device=args.gpu))
trainer.extend(extensions.snapshot_object(CLS,
    filename="CLS_epoch_{.updater.epoch}"), trigger=(args.snapshot, 'epoch'))
#trainer.extend(extensions.ExponentialShift('lr', 0.1), trigger=(30, 'epoch'))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'main/acc', 'val/loss',
    'val/acc', 'elapsed_time', 'lr']))
trainer.extend(extensions.ProgressBar())

trainer.run()
del trainer


#GPU:0
#minibatch-size:100
#epoch:30

Loading
Loaded
epoch       main/loss   main/acc    val/loss    val/acc     elapsed_time  lr        
     total [..................................................]  0.99%
this epoch [##############....................................] 29.76%
       100 iter, 0 epoch / 30 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.98%
this epoch [#############################.....................] 59.52%
       200 iter, 0 epoch / 30 epochs
    33.506 iters/sec. Estimated time to finish: 0:04:54.876108.
     total [#.................................................]  2.98%
this epoch [############################################......] 89.29%
       300 iter, 0 epoch / 30 epochs
    33.499 iters/sec. Estimated time to finish: 0:04:51.947015.
1           0.0667201   0.981608    0.053135    0.983215    10.8158                   
     total [#........................................

30epochだけ学習しました．Predictしていきましょう．

In [0]:
def Load_testDataset(withlabel=False, conv=True):
    input = pd.read_csv("test.csv")
    data = input.values
    #画像とラベルに分ける．画像は255で割って0~1に正規化
    img = (data/255.0).astype(np.float32)
    #imgを28x28にreshape
    if conv:
        img = np.reshape(img, (-1, 1, 28, 28))
    #tupledatasetを作る
    test = tuple_dataset.TupleDataset(img,)
    return test


まずはmnistデータセットを使って正答率を見ます．

In [73]:
#load model
serializers.load_npz('result/b{}/CLS_epoch_{}'.format(args.batchsize,
args.epoch), CLS)
#dataset
_,test = chainer.datasets.get_mnist(withlabel=True, ndim=3)
success = 0
fail = 0
#calc
for i in range(1000):
    x,t = test[i]
    CLS.to_cpu()
    with chainer.using_config('train', False), \
        chainer.using_config('enable_backprop', False):
        y = CLS(x[None, ...]).data.argmax(axis=1)[0]
    #z = model.predictor(x[None, ...]).data#softmax全体を表示
    if(y==t):
        success+=1
    else:
        fail+=1
    #progress
    if(i%100==0):
        print(i)
#out
print("result: success={0}, fail={1}".format(success,fail))
print("ratio = ",success/1000)


0
100
200
300
400
500
600
700
800
900
result: success=996, fail=4
ratio =  0.996


最後に提出用のcsvファイルを作成します

In [0]:
def Predict_num():
    serializers.load_npz('result/b{}/CLS_epoch_{}'.format(args.batchsize,
    args.epoch), CLS)
    test = Load_testDataset()
    test_labels = np.zeros(1).astype(np.int32)
    for i in range(0, 28000):
        x = test[i][0]
        #plt.imshow(x.reshape(28,28), cmap='gray')
        #plt.show()
        with chainer.using_config('train', False), \
            chainer.using_config('enable_backprop', False):
            y = CLS(x[None, ...]).data.argmax(axis=1)[0]
        test_labels = np.append(test_labels, y)
        if(i%1000==0):
            print(i)
        #print("predict:", y)
    test_labels = test_labels[1:]
    return test_labels


コメントアウトしているpltとprintを使うと画像と推論されたラベルがわかる．

In [75]:
test_labels = Predict_num()
submmission = pd.DataFrame(data={'ImageId':(np.arange(test_labels.shape[0])+1),
    'Label':test_labels})
submmission.to_csv('submission.csv', index=False)
submmission.tail()


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2


終わりです